# FewShotPromptTemplate and Fewshot Learning
- Fewshot means you are going to give an example to models to lead better answer.
- Instead of delivering prompt every single time, it is more efficient to give an example to model what I want.

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variable=["country"],
# )
# t.format(country="France")


t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country="France") 


'What is the capital of France'

In [2]:
examples = [
    {
    "question": "What do you know about France?",
    "answer":"""
    Here is what I know:
    Capital:Paris
    Language:French
    Food: Wine and Cheese
    Currency:Euro
    """,
    },
    {"question": "What do you know about Italy?",
     "answer": """
     I know this:
     Capital:Rome
     Language: Italian
     Food: Pizza and Pasta
     Curency: Euro
     """,
     },
    {"question": "What do you know about Greece?",
     "answer": """
     I know this:
     Capital: Athens
     Language:Greek
     Food: Souvlaki and Feta Cheese
     Currency:Euro
     """,
     },
]

In [3]:
# answer with Fewshot
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:

1. Capital: The capital city of France is Paris, which is also its largest city.

2. Language: The official language is French, and it is spoken by the majority of the population.

3. Geography: France shares borders with several countries, including Belgium, Luxembourg, Germany, Switzerland, Italy, Spain, and Andorra. It also has coastlines along the Mediterranean Sea, the Atlantic Ocean, and the English Channel.

4. History: France has a long and influential history, including the French Revolution in 1789, which led to the end of the monarchy and the rise of the First French Republic. It has also been involved in various conflicts, including World War I and World War II.

5. Culture: France is renowned for its cultural contributions, including its cuisine, fashion, art, literature, and cinema. It is ho

'France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:\n\n1. Capital: The capital city of France is Paris, which is also its largest city.\n\n2. Language: The official language is French, and it is spoken by the majority of the population.\n\n3. Geography: France shares borders with several countries, including Belgium, Luxembourg, Germany, Switzerland, Italy, Spain, and Andorra. It also has coastlines along the Mediterranean Sea, the Atlantic Ocean, and the English Channel.\n\n4. History: France has a long and influential history, including the French Revolution in 1789, which led to the end of the monarchy and the rise of the First French Republic. It has also been involved in various conflicts, including World War I and World War II.\n\n5. Culture: France is renowned for its cultural contributions, including its cuisine, fashion, art, literature, and cinem

In [4]:
# Need to format an example

#example_template = """
#    Human: {question}
#    AI: {answer}
#"""

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

# prompt.format(country="Germany")

# now put it inside of the chain
chain = prompt | chat

chain.invoke({
    "country":"Germany"
})

AI: 
     I know this:
     Capital: Berlin
     Language: German
     Food: Bratwurst and Sauerkraut
     Currency: Euro

AIMessageChunk(content='AI: \n     I know this:\n     Capital: Berlin\n     Language: German\n     Food: Bratwurst and Sauerkraut\n     Currency: Euro')

In [5]:
examples2 = [
    {
    "country": "France",
    "answer":"""
    Here is what I know:
    Capital:Paris
    Language:French
    Food: Wine and Cheese
    Currency:Euro
    """,
    },
    {"country": "Italy",
     "answer": """
     I know this:
     Capital:Rome
     Language: Italian
     Food: Pizza and Pasta
     Curency: Euro
     """,
     },
    {"country": "Greece",
     "answer": """
     I know this:
     Capital: Athens
     Language:Greek
     Food: Souvlaki and Feta Cheese
     Currency:Euro
     """,
     },
]

In [6]:
example2_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}? "),
    ("ai", "{answer}"),
])

example2_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example2_prompt,
    examples=examples2,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answer"),
    example2_prompt,
    ("human", "what do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke({"country":"Germany"})

I know this:
     Capital: Berlin
     Language: German
     Food: Bratwurst and Sauerkraut
     Currency: Euro

AIMessageChunk(content='I know this:\n     Capital: Berlin\n     Language: German\n     Food: Bratwurst and Sauerkraut\n     Currency: Euro')

### How to make a dynamic example selector
- this will format your example, see how long the are 
- depending on your setting, it is going to choose example that fit on the prompt

In [7]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

# examples from above cell
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n    Here is what I know:\n    Capital:Paris\n    Language:French\n    Food: Wine and Cheese\n    Currency:Euro\n    \n\nHuman: What do you know about Italy?\nAI: \n     I know this:\n     Capital:Rome\n     Language: Italian\n     Food: Pizza and Pasta\n     Curency: Euro\n     \n\nHuman: What do you know about Greece?\nAI: \n     I know this:\n     Capital: Athens\n     Language:Greek\n     Food: Souvlaki and Feta Cheese\n     Currency:Euro\n     \n\nHuman: What do you know about Brazil?'

In [8]:
from typing import Any, Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector


# this is a very rough simple example
# from list of example above, choose one randomly
class RandomExampleSelector(BaseExampleSelector):
    
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)] # select_examples should return a list!
    
    
    
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n     I know this:\n     Capital:Rome\n     Language: Italian\n     Food: Pizza and Pasta\n     Curency: Euro\n     \n\nHuman: What do you know about Brazil?'

### Serialization
- Saving and loading

### Composition
- How to have various tiny prompt template and put them all together

- How to load prompt templates from the disk

In [11]:
from langchain.prompts import load_prompt


prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

- This allow us to combine many prompts together.

In [12]:
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrrg! That is a secret!! Arg arg!!",
    "question":"What is your fav food?",
})


Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate of rum-infused shrimp! The taste of those succulent crustaceans be like a party in me mouth, matey! Arg arg!")